In [1]:
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

# General imports
import os
import json
import numpy as np
from datetime import datetime, timedelta


# custom module
from susy_qm import calculate_Hamiltonian

from qiskit_ibm_runtime import QiskitRuntimeService, Session, SamplerV2 as Sampler, EstimatorV2 as Estimator
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.primitives import StatevectorEstimator

In [2]:
potential = 'QHO'
cut_off = 2
shots = 1024

In [3]:
H = calculate_Hamiltonian(cut_off, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])
min_eigenvalue = min(eigenvalues.real)

#create qiskit Hamiltonian Pauli string
hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [4]:
estimator = StatevectorEstimator()

qc = QuantumCircuit(num_qubits)
#qc.ry(Parameter('theta'), 0)
qc.ry(Parameter('theta'), 1)
qc.draw("mpl", style="iqp")

parameter_values = [np.pi]

pm = generate_preset_pass_manager(optimization_level=1)
isa_circuit = pm.run(qc)
isa_observable = hamiltonian.apply_layout(isa_circuit.layout)

job = estimator.run([(isa_circuit, isa_observable, parameter_values)])
result = job.result()

print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

 > Expectation value: 0.0
 > Metadata: {'target_precision': 0.0, 'circuit_metadata': {}}


In [5]:
service = QiskitRuntimeService()
backend = service.backend('ibm_brisbane')

In [12]:
'''
qc = QuantumCircuit(num_qubits)
qc.ry(Parameter('theta'), 1)
qc.draw("mpl", style="iqp")

pm = generate_preset_pass_manager(optimization_level=1, target=backend.target)
isa_circuit = pm.run(qc)
isa_observable = hamiltonian.apply_layout(isa_circuit.layout)

with Session(backend=backend) as session:
    estimator = Estimator(mode=session, options={"default_shots": 1024})

    job = estimator.run([(isa_circuit, isa_observable, parameter_values)])
    result = job.result()
'''

In [8]:
pm = generate_preset_pass_manager(optimization_level=1, target=backend.target)
isa_circuit = pm.run(qc)
isa_observable = hamiltonian.apply_layout(isa_circuit.layout)

In [9]:
def cost_function(params, isa_circuit, isa_observable):
   
    estimator = Estimator(mode=backend, options={"default_shots": 1024})
    result = estimator.run([(isa_circuit, isa_observable, params)]).result()
    energy = result[0].data.evs

    return energy


def wrapped_cost(params):
    #print("Session status:", session.active)
    energy = cost_function(params, isa_circuit, isa_observable)
    print("Energy:", energy)
    return energy

In [10]:
# VQE
vqe_start = datetime.now()

#variables
num_vqe_runs = 1
max_iter = 6
strategy = "randtobest1bin"
tol = 1e-2
abs_tol = 1e-2
popsize = 5

# Generate Halton sequence
num_dimensions = 1
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]

run_start = datetime.now()

res = differential_evolution(wrapped_cost,
                            bounds=bounds,
                            maxiter=max_iter,
                            tol=tol,
                            atol=abs_tol,
                            strategy=strategy,
                            popsize=popsize,
                            init=scaled_samples
                        )

if res.success == False:
        print("Not converged")

energies.append(res.fun)
x_values.append(res.x)
success.append(res.success)
num_iters.append(res.nit)
num_evaluations.append(res.nfev)

run_end = datetime.now()
run_time = run_end - run_start
run_times.append(run_time)

total_run_time = sum(run_times, timedelta())
vqe_end = datetime.now()
vqe_time = vqe_end - vqe_start

Energy: 1.0006359704909693
Energy: 0.004280056215663741
Energy: 0.4356732610082961
Energy: 0.5611659764256133
Energy: 0.7883158229863101
Energy: 0.4715374841168996
Energy: 0.42051461690338193
Energy: -0.0005723006486073601
Energy: 0.27017922969365704
Energy: 0.3544066178810054
Energy: 0.010837094006228953
Energy: 0.36890243902439024
Energy: 0.7622284334900267
Energy: 0.0071428571428571175
Energy: 0.16497571505666486
Energy: 0.04850746268656714
Energy: 0.038700801531283635
Energy: 0.057463042441583234
Energy: 0.11283445261699176
Energy: -0.12511291779584466
Energy: 0.038926681783824635
Energy: -0.09130736531621408
Energy: 0.17209593699659925
Energy: 0.19892134477600187
Energy: 0.20573990444400814
Energy: 0.03299844101853455
Energy: 0.11139387445307619
Energy: -0.1412290222927277
Energy: 0.0020131654630280438


c:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\.venv\Lib\site-packages\qiskit_ibm_runtime\qiskit_runtime_service.py:904: UserWarning: Your current pending jobs are estimated to consume 11.096129969706686 quantum seconds, but you only have 1 quantum seconds left in your monthly quota; therefore, it is likely this job will be canceled
  warnings.warn(warning_message)


Energy: 0.19585798816568045


: 

In [ ]:
# VQE
vqe_start = datetime.now()

#variables
num_vqe_runs = 1
max_iter = 10
strategy = "randtobest1bin"
tol = 1e-2
abs_tol = 1e-2
popsize = 5

# Generate Halton sequence
num_dimensions = 1
num_samples = popsize
halton_sampler = Halton(d=num_dimensions)
halton_samples = halton_sampler.random(n=num_samples)
scaled_samples = 2 * np.pi * halton_samples

#data arrays
energies = []
x_values = []
success = []
run_times = []
num_iters = []
num_evaluations = []

#Optimizer
bounds = [(0, 2 * np.pi) for _ in range(num_dimensions)]

with Session(backend=backend) as session:

    estimator = Estimator(mode=session, options={"default_shots": 1024})

    def wrapped_cost(params):
        print("Session status:", session.active)
        energy = cost_function(params, isa_circuit, isa_observable)
        print("Energy:", energy)
        return energy
    
    run_start = datetime.now()

    res = differential_evolution(wrapped_cost,
                                bounds=bounds,
                                maxiter=max_iter,
                                tol=tol,
                                atol=abs_tol,
                                strategy=strategy,
                                popsize=popsize,
                                init=scaled_samples
                            )
    
    if res.success == False:
            print("Not converged")

    energies.append(res.fun)
    x_values.append(res.x)
    success.append(res.success)
    num_iters.append(res.nit)
    num_evaluations.append(res.nfev)

    run_end = datetime.now()
    run_time = run_end - run_start
    run_times.append(run_time)

total_run_time = sum(run_times, timedelta())
vqe_end = datetime.now()
vqe_time = vqe_end - vqe_start

In [ ]:
job = service.job('d0gweh236cs0008rcnzg')
job_result = job.result()

for idx, pub_result in enumerate(job_result):
    print(f"Expectation values for pub {idx}: {pub_result.data.evs}")

In [ ]:
res

In [ ]:
# Save run
run = {
    "starttime": str(vqe_start),
    "potential": potential,
    "cutoff": cut_off,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "num_VQE": num_vqe_runs,
    "shots": shots,
    "Optimizer": {
        "name": "differential_evolution",
        "bounds": "[(0, 2 * np.pi) for _ in range(np.prod(params_shape))]",
        "maxiter": max_iter,
        "tolerance": tol,
        "abs_tolerance": abs_tol,
        "strategy": strategy,
        "popsize": popsize,
        'init': 'scaled_samples',
    },
    "results": energies,
    "params": [x.tolist() for x in x_values],
    "num_iters": num_iters,
    "num_evaluations": num_evaluations,
    "success": np.array(success, dtype=bool).tolist(),
    "run_times": [str(x) for x in run_times],
    "parallel_run_time": str(vqe_time),
    "total_VQE_time": str(total_run_time),
    "total_device_time": str(device_time),
    "Backend": backend.name
}

run

In [64]:
# Save the variable to a JSON file
base_path = r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\VQE\Differential Evolution\Real Device\Files\\"
path = base_path + "{}_{}.json".format(potential, cut_off)
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)